In [1]:
!pip install langchain elasticsearch langchain-community langchain-elasticsearch python-dotenv sentence-transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.3/906.3 kB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.9/438.9 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 49.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 49.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/2

In [7]:
!pip install -U langchain-openai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.0/69.0 kB 5.8 MB/s eta 0:00:00


In [2]:
from google.colab import files
uploaded = files.upload()
file_path = list(uploaded.keys())[0]


Saving BMC.txt to BMC.txt
Saving Cleveland.txt to Cleveland.txt
Saving Forbes.txt to Forbes.txt
Saving Mayoclinic.txt to Mayoclinic.txt
Saving wikipedia.txt to wikipedia.txt


In [12]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings

# Load all uploaded files
all_documents = []

for file_name in uploaded.keys():
    loader = TextLoader(file_name)
    docs = loader.load()
    all_documents.extend(docs)

# Split into chunks
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=100)
docs = text_splitter.split_documents(all_documents)

# Print filenames to confirm multiple sources
print("📄 Loaded files:")
for doc in docs[:10]:  # Preview 10 chunks
    print("-", doc.metadata['source'])

# Embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


📄 Loaded files:
- BMC.txt
- BMC.txt
- BMC.txt
- BMC.txt
- BMC.txt
- Cleveland.txt
- Cleveland.txt
- Cleveland.txt
- Cleveland.txt
- Cleveland.txt


In [ ]:
from elasticsearch import Elasticsearch
from langchain_elasticsearch import ElasticsearchStore

cloud_id = "4f49......" #removed my cloud id
es_user = "elastic"
es_pass = "Password" #removed my password

es_client = Elasticsearch(
    cloud_id=cloud_id,
    basic_auth=(es_user, es_pass)
)

vectorstore = ElasticsearchStore.from_documents(
    documents=docs,
    embedding=embeddings,
    es_connection=es_client,
    index_name="agentic-docs"
)


In [21]:
from langchain.tools import Tool

retriever_tool = Tool(
    name="document_retrieval",
    description="Use this to search for relevant documents in the knowledge base.",
    func=vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 4}).get_relevant_documents
)


In [ ]:
from langchain_openai import ChatOpenAI
import os

os.environ["OPENAI_API_KEY"] = "sk......" #removed my openai key 

llm = ChatOpenAI(temperature=0.7, model_name="gpt-3.5-turbo")


In [26]:
from langchain.agents import initialize_agent
from langchain.agents.agent_types import AgentType

agent = initialize_agent(
    tools=[retriever_tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,  # decision-making agent
    verbose=True  # shows internal reasoning
)


In [27]:
response = agent.run("How can AI assist in patient monitoring and chronic illness?")
print("\n Agent Response:")
print(response)




> Entering new AgentExecutor chain...
I should use the document retrieval tool to search for relevant information on how AI can assist in patient monitoring and chronic illness.
Action: document_retrieval
Action Input: query='AI assist in patient monitoring and chronic illness'
Observation: [Document(metadata={'source': 'Mayoclinic.txt'}, page_content='How can AI technology advance medicine and public health?\nWhen it comes to supporting the overall health of a population, AI can help people manage chronic illnesses themselves — think asthma, diabetes and high blood pressure — by connecting certain people with relevant screening and therapy, and reminding them to take steps in their care, such as take medication.'), Document(metadata={'source': 'BMC.txt'}, page_content='Results\nIntegrating AI into healthcare holds excellent potential for improving disease diagnosis, treatment selection, and clinical laboratory testing. AI tools can leverage large datasets and identify patterns to su

In [28]:
response = agent.run("How can AI assist in patient monitoring?")
print("\n Agent Response:")
print(response)



> Entering new AgentExecutor chain...
I should use the document_retrieval function to search for relevant information on how AI can assist in patient monitoring.
Action: document_retrieval
Action Input: query='AI in patient monitoring'
Observation: [Document(metadata={'source': 'wikipedia.txt'}, page_content='Artificial intelligence in healthcare is the application of artificial intelligence (AI) to analyze and understand complex medical and healthcare data. In some cases, it can exceed or augment human capabilities by providing better or faster ways to diagnose, treat, or prevent disease.'), Document(metadata={'source': 'BMC.txt'}, page_content='Research Significance\nThis review article provides a comprehensive and up-to-date overview of the current state of AI in clinical practice, including its potential applications in disease diagnosis, treatment recommendations, and patient engagement. It also discusses the associated challenges, covering ethical and legal considerations and t